# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [3]:
import pandas as pd
from IPython.display import display
data = pd.read_csv('/datasets/data.csv')
#print(data)
data.info()
data['children'] = abs(data['children'])

for row in range(len(data)):
    if data.loc[row, 'children'] == 20:
        data.loc[row, 'children'] = 2 #убираем артефакт в столбце с количеством детей. Скорее всего 20 детей - это ошибка
display(data)#менеджера или умышленное завышение соискателем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

Данные подлежат комплексной обработке, встречаются артефакты почти во всех колонках. Данные колонки со стажем работы и заработком подлежат корректировке(содержатся пропуски, а общий стаж нам интересен в целых днях, всё, что после запятой, нас не интересует). Учитывая, что отсутствующих значений по стажу и заработку одинаковое количество, можно предположить, что эти пропуски относятся к одним и тем же заёмщикам. Колонка education заполнена с разным регистром, что так же подлежит корректировке. Особое внимание стоит уделить колонке со стажем, почему-то пенсионеры имеют неестественно огромный стаж, а работающие - отрицательный.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#print(data.sort_values(by = 'days_employed').reset_index(drop = True).loc[:15906, 'days_employed'])

# Строка, в которой стаж из
# отрицательного превращается в положительный найдена эмпирически. Теперь мы знаем, что артефакты - стаж пенсионеров,
# нулевого стажа ни у кого нет

days_employed_median = data.loc[:15905, 'days_employed'].median() #значение, которым будем заполнять пропуски

# если потенциальный заёмщик пенсионер, то такое медианное значение стажа ему не подойдет. Как и значения стажа
# остальных пенсионеров - для корректного анализа их необходимо заменить. Будем опираться на среднее время работы мужчин
# и женщин - 41,5 и 36,5 лет соответственно, или 15147 и 13322 дней.

total_income_median = data.sort_values(by = 'total_income').reset_index(drop = True).loc[:19351, 'total_income'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
for row in range(len(data)):
    if data.loc[row, 'days_employed'] > 0:
        if data.loc[row, 'gender'] == 'M':
            data.loc[row, 'days_employed'] = 15147
        if data.loc[row, 'gender'] == 'F':
            data.loc[row, 'days_employed'] = 13322
        if abs(data.loc[row, 'days_employed']/365) > data.loc[row, 'dob_years']: #если встречаются значения стажа большие, 
            #чем возраст соискателя то мы меняем это значение на медианное
            data.loc[row, 'days_employed'] = days_employed_median
data['days_employed'] = abs(data['days_employed'])

data['total_income'] = data['total_income'].fillna(total_income_median)
data.info()
#print(data['days_employed'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Пропущенные значения в колонке стажа заменены на медианное для соискателей - работников. Для пенсионеров в этой колонке присутствует особенность - их стаж указан слишком большим числом, поэтому я заменил его на среднее значение стажа для мужчин и женщин соответственно. Остальные значения стажа указаны со знаком минус, поэтому их я заменил на аналогичные по модулю значения, так как рабочий стаж отрицательным быть не может. В колонке с доходом отсутствующие значения я заменил на медианное. Медианное значение выбрано как наиболее подходящее для анализа заработка, так как среднее значение не всегда будет корректно отображать ситуацию. Выбросить пропущенные значения в даннх колонках не представляется возможным, так как пропуски составляют более 10% предоставленной информации. Объяснить отсутствие данных можно двумя способами - первый, и наиболее очевидный, вариант: люди сами предпочли не оставлять эту информацию банку по личным причинам. Однако, не стоит исключать и ошибку менеджера банка при заполнении анкеты клиента, что так же могло привести к пропускам.

### Замена типа данных

In [3]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В колонках стаж и доход данные заменены на целочисленный формат. В данном анализе это применимо к указанным колонкам, потому что нас не интересуют неполные рабочие дни, входящие в стаж и количество копеек в зарплате у потенциального заёмщика. Для замены типа данных я применил метод astype, так как он позволяет перевести весь столбец структуры DataFrame перевести в нужный мне тип данных без использования сложных конструкций и циклов.

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower() #меняем регистр столбца образование для избежания пропусков дубликатов
print(data.duplicated().sum())
data=data.drop_duplicates().reset_index(drop = True)
data.info()

71
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Обнаружен 71 дубликат, который необходимо обработать. В условиях представленной выборки это количество пренебрежимо мало, поэтому проще всего будет их удалить. 
Поиск дубликатов и замена проиходили с помощью встроенных методов pandas - duplicated и drop_duplicates. Чтобы не нарушать индексацию таблицы при удалении дубликатов была указана методом reset_index команда к обновлению индексов сохраненных значений.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
def lemmatization(text):
    lemmas = m.lemmatize(text)
    text = ' '.join(lemmas)
    return text
data['lemmatised_purpose'] = data['purpose'].apply(lemmatization)
#print(data['lemmatised_purpose'].unique())
list_for_count = [string.split(' ') for string in data['lemmatised_purpose'].values] #разбиваем строки в столбце 
# lemmatised_purpose на отдельные значения и формируем "список списков" найденных лемм
def lemmatised_counter(list_of_lemmas): # пишем функцияю, формирующую список, каждая ячейка которого 
    #содержит отдельно взятую лемму
    lemmatised_purposes=[]
    for row in list_of_lemmas:
        for col in row:
            lemmatised_purposes.append(col)
    return lemmatised_purposes
lemmatised_list = lemmatised_counter(list_for_count)
for row in range(len(lemmatised_list)):
    if lemmatised_list[row] == 'жилье':
        lemmatised_list[row] = 'недвижимость'

summ_of_lemmas = Counter(lemmatised_list) # считаем количество лемм

for row in range(len(data)):
    if "жилье" or "недвижимость" in data.loc[row, 'lemmatised_purpose']:
        data.loc[row, 'purpose_lemm'] = 'недвижимость'
    if "автомобиль" in data.loc[row, 'lemmatised_purpose']:
        data.loc[row, 'purpose_lemm'] = 'автомобиль'
    if "образование" in data.loc[row, 'lemmatised_purpose']:
        data.loc[row, 'purpose_lemm'] = 'образование'
    if "свадьба" in data.loc[row, 'lemmatised_purpose']:
        data.loc[row, 'purpose_lemm'] = 'свадьба'

**Вывод**

Больше всего кредитов взяли на недвижимоть - 10811 запросов. Меньше всего на ремонт 607. Также, берут кредиты на покупку автомобиля - 4306, получение образования - 4013, и свадьбу - 2324 запроса.Эти данные получены после процесса лемматизации. Для этого я подключил библиотеку pymystem3 и с её помощью функцией lemmatization разбил сточки столбца целей на леммы. Далее, я сгенерировал список, каждая ячейка которого содержит список лемм, выделенных ранее. После этого, функцией lemmatised_counter я составил одномерный список лемм, каждая ячейка которого содержит только одну выделенную лемму. Это было необходимо для использования метода Counter, который посчитал количество лемм в столбце целей. Стоит обратить внимание, что лемма "недвижимоть" и лемма"жилье" отражает одну цель, а значит их стоит объеденить, что я и сделал через цикл for. После этого, в data я создал столбец 'purpose_lemm', в который записал своеобразный id цели для более простой категоризации целей на кредит.



### Категоризация данных

In [11]:
for row in range(len(data)): # классифицируем доход по уровню для упрощения анализа
    if data.loc[row, 'total_income'] < 50000:
        data.loc[row, 'total_income_id'] = 'низкий'
    if 50000 <= data.loc[row, 'total_income'] < 120000:
        data.loc[row, 'total_income_id'] = 'средний'
    if data.loc[row, 'total_income'] >= 120000:
        data.loc[row, 'total_income_id'] = 'высокий'

data_pivot = data.pivot_table(index = ['children', 'total_income_id', 'family_status', 'purpose_lemm'],
                        columns = ['debt'], values = ['purpose'], fill_value = 'данных не обнаружено', aggfunc = 'count')
#формируем сводную таблицу по интересующим нас категориям и считаем количество значений. Для красоты таблицы я заменил 
# NaN на "данных не обнаружено"
data_pivot.tail(30)


purpose  \
debt                                                                            0   
children total_income_id family_status         purpose_lemm                         
3        средний         в разводе             автомобиль                       1   
                                               недвижимость                     2   
                         вдовец / вдова        недвижимость                     1   
                         гражданский брак      автомобиль                       1   
                                               недвижимость                     4   
                                               свадьба                          3   
                         женат / замужем       автомобиль                      10   
                                               недвижимость                    45   
                                               образование                      8   
4        высокий         Не женат / не замужем образование                      1   
                         в разводе             автомобиль                       1   
                         вдовец / вдова        недвижимость                     1   
                         гражданский брак      автомобиль                       1   
                                               недвижимость                     3   
                                               свадьба                          2   
                         женат / замужем       автомобиль                       5   
                                               недвижимость                    11   
                                               образование                      5   
         средний         Не женат / не замужем недвижимость  данных не обнаружено   
                         гражданский брак      недвижимость                     1   
                                               свадьба                          1   
                         женат / замужем       автомобиль                       2   
                                               недвижимость                     2   
                                               образование                      1   
5        высокий         гражданский брак      автомобиль                       1   
                                               свадьба                          1   
                         женат / замужем       недвижимость                     4   
                                               образование                      1   
         низкий          женат / замужем       недвижимость                     1   
         средний         женат / замужем       автомобиль                       1   

                                                                                   
debt                                                                            1  
children total_income_id family_status         purpose_lemm                        
3        средний         в разводе             автомобиль    данных не обнаружено  
                                               недвижимость  данных не обнаружено  
                         вдовец / вдова        недвижимость  данных не обнаружено  
                         гражданский брак      автомобиль    данных не обнаружено  
                                               недвижимость  данных не обнаружено  
                                               свадьба                          3  
                         женат / замужем       автомобиль                       2  
                                               недвижимость                     3  
                                               образование   данных не обнаружено  
4        высокий         Не женат / не замужем образование   данных не обнаружено  
                         в разводе             автомобиль    данных не обнаружено  
                         вдовец / вдова        недвижимость  данных не обнаружено  
        

**Вывод**

Удивительно, но кредит на недвижимость чаще всего берут холостые люди с высоким уровнем дохода и без детей.
Процесс написания кода описан в комментариях к нему. Категоризация происходила по столбцам с количеством детей, семейным статусом, "идентификатором" уровня дохода, семейному положению и лемматизированной цели. В колонках я указал только идентификатор должника, так как это наиболее интересующая нас информация. Величина агрегации - цель, функция, применяема к агргации - count. Таким образом, я считаю количество отдельно взятых целей и отслеживаю по строкам, указанным в index зависимости от интересующих меня параметров надежности потенциального заемщика.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
#print(data.groupby('debt')['children'].value_counts())
data_pivot_children = data.pivot_table(index = ['children'],
                        columns = ['debt'], values = ['purpose'], fill_value = 0,
                                       aggfunc = {'count'})
data_pivot_children['counted_summ'] = data.groupby('children')['debt'].count()
data_pivot_children['counted_percent,%'] = (data_pivot_children.iloc[:, 1]/data_pivot_children.iloc[:, 2])*100
print(data_pivot_children)
def children_count(children):
    if children > 0:
        return 1
    else:
        return 0
def children_debt_count(children):
    if children > 0:
        if debt >0:
            return 1
    else:
        return 0

children_in_data = data['children'].apply(children_count).value_counts()
children_in_data_with_debts = data_pivot_children.iloc[1:,1].sum()
print('семей с детьми', children_in_data.loc[1])
print('должников с детьми {:.2%}'.format(children_in_data_with_debts/children_in_data.loc[1]))
print('должников без детей {:.2f}%'.format(data_pivot_children.iloc[0, 3]))

         purpose       counted_summ counted_percent,%
           count                                     
debt           0     1                               
children                                             
0          13028  1063        14091          7.543822
1           4410   445         4855          9.165808
2           1926   202         2128          9.492481
3            303    27          330          8.181818
4             37     4           41          9.756098
5              9     0            9          0.000000
семей с детьми 7363
должников с детьми 9.21%
должников без детей 7.54%


**Вывод**

Вне зависимости от количества детей потенциального соискателя, процент должников колеблется в районе 7.5 - 10%. Единственным исключением являются соискатели с 5ю детьми, среди них должников нет вообще. Однако, таких семей всего 5. С одной стороны, можно утверждать, что количество детей не влияет на возврат кредита, и рассматривать соискателей с 5ю детьми как артефакт. С другой стороны, экстраполируя эту динамику, можно утверждать, что многодетные семьи чаще возвращают кредиты. Однако, в условиях данной задачи, на мой взгляд, правильнее будет пренебречь этими данными. Также, прослеживается тенденция к тому, что соискатели с детьми чаще становятся должниками. Но и в этом случае, колебания величин находятся в пределах погрешности.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#print(data.groupby('debt')['family_status_id'].value_counts())
data_pivot_family = data.pivot_table(index = ['family_status_id'],
                        columns = ['debt'], values = ['purpose'], fill_value = 0,
                                       aggfunc = {'count'})
data_pivot_family['counted_summ'] = data.groupby('family_status_id')['debt'].count()
data_pivot_family['counted_percent,%'] = (data_pivot_family.iloc[:, 1]/data_pivot_family.iloc[:, 2])*100
data_pivot_family

purpose      counted_summ counted_percent,%
                   count                                    
debt                   0    1                               
family_status_id                                            
0                  11408  931        12339          7.545182
1                   3763  388         4151          9.347145
2                    896   63          959          6.569343
3                   1110   85         1195          7.112971
4                   2536  274         2810          9.750890

**Вывод**

Процент должников для каждой категории соискателей колеблется от 7,5 до 10 %, что позволяет сделать вывод об отсутствии заивисмости между семейным положением и любви общения с коллекторами =)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
#print(data.groupby('debt')['total_income_id'].value_counts())
data_pivot_income = data.pivot_table(index = ['total_income_id'],
                        columns = ['debt'], values = ['purpose'], fill_value = 0,
                                       aggfunc = {'count'})
data_pivot_income['counted_summ'] = data.groupby('total_income_id')['debt'].count()
data_pivot_income['counted_percent,%'] = (data_pivot_income.iloc[:, 1]/data_pivot_income.iloc[:, 2])*100
data_pivot_income

purpose       counted_summ counted_percent,%
                  count                                     
debt                  0     1                               
total_income_id                                             
высокий           13419  1190        14609          8.145664
низкий              349    23          372          6.182796
средний            5945   528         6473          8.156960

**Вывод**

Тенденция должников убывает с падением доходов(что удивительно), но не значительно, примерно с 6% до 8%. Однако, эти данные зависят от определения группы дохода и их колебания происходят в пределах погрешности выборки, так что можно утверждать, что зависимость так же отсутствует.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#print(data.groupby('debt')['purpose_lemm'].value_counts())
data_pivot_purpose = data.pivot_table(index = ['purpose_lemm'],
                        columns = ['debt'], values = ['purpose'], fill_value = 0,
                                       aggfunc = {'count'})
data_pivot_purpose['counted_summ'] = data.groupby('purpose_lemm')['debt'].count()
data_pivot_purpose['counted_percent,%'] = (data_pivot_purpose.iloc[:, 1]/data_pivot_purpose.iloc[:, 2])*100
data_pivot_purpose

purpose      counted_summ counted_percent,%
               count                                    
debt               0    1                               
purpose_lemm                                            
автомобиль      3903  403         4306          9.359034
недвижимость   10029  782        10811          7.233373
образование     3643  370         4013          9.220035
свадьба         2138  186         2324          8.003442

**Вывод**

Колебания величин происходят в районе 7-9%. Автомодильные кредиты и кредиты на образование должники берут чаще, однако такое изменение величин можно списать на погрешность.

## Шаг 4. Общий вывод

Нет прямых зависимостей между отдельно взятыми параметрами и тем, как соискатель будет выплачивать кредит, колебания сравнительных величин находятся в пределах статистической погрешности. Чаще всего кредит берут холостые, бездетные, с высоким достатком и на недвижимость, реже всего многодетные на цели, не связаные с покупкой недвижимости. А многодетные семьи ( 5 человек) и вовсе ни разу не были должниками.
